In [1]:
import os  
import shutil  
import torch  
import collections  
from torchvision import transforms,datasets  
from __future__ import print_function, division  
import os  
import torch  
import pylab  
import pandas as pd  
import torch.nn as nn  
import torch.nn.functional as F  

from torch.autograd import Variable  
from skimage import io, transform  
import numpy as np  
import matplotlib.pyplot as plt  
from torch.utils.data import Dataset, DataLoader  
from torchvision import transforms, utils  
import math  
from PIL import Image 

# Ignore warnings  
import warnings  
warnings.filterwarnings("ignore")  
  
plt.ion()   # interactive mode  
import os
print(os.listdir("./data"))
from torch.utils.data import DataLoader
# Any results you write to the current directory are saved as output.

['test', 'test.zip', 'train', 'train.zip']


In [2]:
len(os.listdir("./data/train"))

25000

In [3]:
import torch

In [4]:
import os
from PIL import  Image
from torch.utils import data
import numpy as np
from torchvision import  transforms as T


class DogCat(data.Dataset):
    
    def __init__(self,root,transforms=None,train=True,test=False):
        '''
        主要目标： 获取所有图片的地址，并根据训练，验证，测试划分数据
        '''
        self.test = test
        self.transforms =transforms
        imgs = [os.path.join(root,img) for img in os.listdir(root)] 

        # test1: data/test1/8973.jpg
        # train: data/train/cat.10004.jpg 
        if self.test:
            imgs = sorted(imgs,key=lambda x:int(x.split('.')[-2].split('/')[-1]))
        else:
            imgs = sorted(imgs,key=lambda x:int(x.split('.')[-2]))
            
        imgs_num = len(imgs)

        if self.test:
            self.imgs = imgs
        elif train:
            self.imgs = imgs[:int(0.7*imgs_num)]
        else :
            self.imgs = imgs[int(0.7*imgs_num):]
            
    
        if transforms is None:
            normalize = T.Normalize(mean = [0.485, 0.456, 0.406], 
                                     std = [0.229, 0.224, 0.225])

            if self.test or not train: 
                self.transforms = T.Compose([
                    T.Scale(224),
                    T.CenterCrop(224),
                    T.ToTensor(),
                    normalize
                    ]) 
            else :
                self.transforms = T.Compose([
                    T.Scale(256),
                    T.RandomSizedCrop(224),
                    T.RandomHorizontalFlip(),
                    T.ToTensor(),
                    normalize
                    ]) 
                
        
    def __getitem__(self,index):
        '''
        一次返回一张图片的数据
        '''
        img_path = self.imgs[index]
        if self.test: label = int(self.imgs[index].split('.')[-2].split('/')[-1])
        else: label = 1 if 'dog' in img_path.split('/')[-1] else 0
        data = Image.open(img_path)
        data = self.transforms(data)
        return data, label
    
    def __len__(self):
        return len(self.imgs)

In [5]:
data_transform = transforms.Compose([  
    transforms.Resize(84),    
    transforms.CenterCrop(84),  
    transforms.ToTensor(),  
    transforms.Normalize(mean = [0.5,0.5,0.5],std = [0.5,0.5,0.5])  
])  

In [6]:
dogcat = DogCat("./data/train/",train=True)
test = DogCat("./data/train/",train=False,test=False)
pred = DogCat("./data/test/",test=True,train=False)
print(len(dogcat),len(test),len(pred))

17500 7500 12500


In [7]:
batch_size = 10
train_dataloader = DataLoader(dogcat,batch_size=64,shuffle = True)
valid_dataloader  = DataLoader(test,batch_size=64,shuffle = True)
pred_dataloader = DataLoader(pred,batch_size=64,shuffle = True)

In [8]:
class Net(nn.Module):  
    def __init__(self):  
        super(Net,self).__init__()  
          
        self.conv1 = nn.Conv2d(3,6,5)  
        self.pool = nn.MaxPool2d(2,2)  
        self.conv2 = nn.Conv2d(6,16,5)  
        self.fc1 = nn.Linear(16 * 18 * 18,800)  
        self.fc2 = nn.Linear(800,120)  
        self.fc3 = nn.Linear(120,2)  
          
    def forward(self,x):  
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        x = x.view(-1,16 * 18 * 18)  
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))  
        x = self.fc3(x)  
          
        return x  
  

In [9]:
import torch.optim as optim  
import datetime
from torchvision import models

net = Net()   
net = net.cuda()

cirterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(net.parameters(),lr=0.0001, betas=(0.9, 0.99))  
  

In [10]:
net  = models.vgg16(pretrained=True).cuda()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\win/.torch\models\vgg16-397923af.pth
100.0%


In [16]:

for epoch in range(50):  
    running_loss = 0.0  
      
    for i,data in enumerate(train_dataloader):  
        inputs,labels = data  
        inputs = inputs.cuda()
        labels = labels.cuda()
        #print(type(inputs))
        #print(type(labels))
        inputs,labels = Variable(inputs),Variable(labels) 
        #print(data[1])
        
        optimizer.zero_grad()  
        outputs = net(inputs)  
        loss = cirterion(outputs,labels)  
        loss.backward()  
        optimizer.step()  
          
        running_loss += loss.data[0]  
          
        if i % 200 == 199:  
            nowTime=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')#现在
            print('[%d %5d] loss: %.3f, Time:' % (epoch + 1,i + 1,running_loss / 2000)) 
            print(nowTime)
            running_loss = 0.0
    
print('finished training!')  

[1   200] loss: 0.691, Time:
2018-04-26 00:42:18
[2   200] loss: 0.691, Time:
2018-04-26 00:44:56
[3   200] loss: 0.691, Time:
2018-04-26 00:47:33
[4   200] loss: 0.691, Time:
2018-04-26 00:50:11
[5   200] loss: 0.691, Time:
2018-04-26 00:52:49
[6   200] loss: 0.691, Time:
2018-04-26 00:55:27
[7   200] loss: 0.691, Time:
2018-04-26 00:58:05
[8   200] loss: 0.691, Time:
2018-04-26 01:00:43
[9   200] loss: 0.691, Time:
2018-04-26 01:03:22
[10   200] loss: 0.691, Time:
2018-04-26 01:06:00
[11   200] loss: 0.691, Time:
2018-04-26 01:08:38
[12   200] loss: 0.691, Time:
2018-04-26 01:11:16
[13   200] loss: 0.691, Time:
2018-04-26 01:13:54
[14   200] loss: 0.691, Time:
2018-04-26 01:16:32
[15   200] loss: 0.691, Time:
2018-04-26 01:19:10
[16   200] loss: 0.691, Time:
2018-04-26 01:21:48
[17   200] loss: 0.691, Time:
2018-04-26 01:24:26
[18   200] loss: 0.691, Time:
2018-04-26 01:27:05
[19   200] loss: 0.691, Time:
2018-04-26 01:29:43
[20   200] loss: 0.691, Time:
2018-04-26 01:32:21
[21   200

In [17]:
correct = 0  
total = 0  
  
for data in valid_dataloader:  
    images,labels = data  
    images = images.cuda()
    labels = labels.cuda()
    outputs = net(Variable(images)).cuda()  
    _,predicted = torch.max(outputs.data,1)  
    total += labels.size(0)  
    correct += (predicted == labels).sum()  
  

print('Accuracy of the network on the 5000 test images: %d %%' % (100 * correct / total))  

RuntimeError: cuda runtime error (2) : out of memory at c:\users\administrator\downloads\new-builder\win-wheel\pytorch\aten\src\thc\generic/THCStorage.cu:58